<h1><strong>Taking Screenshots and Downloading PDFs</strong></h1>   
<strong>By: </strong>Jeff Ocampo   

<strong>Created: </strong>May 2020   

<h3><strong>About</strong></h3>   
<strong>Q: </strong>What's this doing?   

<strong>A: </strong>Using the database of machine learning PDFs from Google, take screenshots of those PDFs and download some of them.   

<h3><strong>Steps</strong></h3>   
<ol>
    <li>Get list of links from the SQL Server database</li>
    <li>Use Selenium to run Google Chrome and take screenshots</li>
    <li>Download the first 20 PDFs to a specific folder</li>
</ol>   
<h3><strong>Python</strong></h3>   
    



<strong>Packages</strong>

In [ ]:
from selenium import webdriver
import pyodbc
import getpass
import time
import wget

<strong>Get links from SQL Server</strong>

In [ ]:
# Hey, I just met you, and this is crazy, but here's my server name, so call me, maybe?
# ...not showing you this either
sql_server = getpass.getpass('SQL Sever name: ')

In [ ]:
sql_conn = pyodbc.connect('Driver={SQL Server};Server='+sql_server+';Database=GOOGLE_FILES;Trusted_Connection=yes;')
sql_cursor = sql_conn.cursor()

In [ ]:
search_results_query = 'SELECT ID, LINK FROM DBO.SEARCH_RESULTS'

In [ ]:
sql_cursor.execute(search_results_query)
ml_pdfs = [item for item in sql_cursor.fetchall()]

<strong>Grab screenshots</strong>

In [ ]:
def Load_N_Screenshot(link, file_name):
    """
    Opens a new Chrome browser window, loads the link, takes a screenshot, saves it, and closes. 
    """
    sel_options = webdriver.ChromeOptions()
    sel_options.add_argument('--ignore-certificate-errors')
    sel_options.add_argument("--test-type")
    driver = webdriver.Chrome(executable_path="c:\drivers\chromedriver.exe",options=sel_options)
    
    driver.get(link)
    
    time.sleep(10) #could use WebDriverWait and look for a page element but, again, just a weekend project
    
    # Might be too lazy to build this to fit different naming conventions
    # This just takes the ID from the database and concat's to a hard coded value
    # It's a weekend project...
    tmp_file_name = 'D:\Projects\Python\ML_PDF\Screenshots\ML_PDF_Screenshot_' + str(file_name) + '.png'
    
    driver.save_screenshot(tmp_file_name)
    
    driver.close()

In [ ]:
for item in ml_pdfs:
    Load_N_Screenshot(item[1], item[0])

<strong>Download the first 20 pdfs</strong>

In [ ]:
for i,item in enumerate(ml_pdfs):
    if i < 20: # don't want to download every PDF for a weekend project
        tmp_file_name = 'D:\Projects\Python\ML_PDF\PDF\ML_PDF_' + str(item[0]) + '.pdf'
        try:
            wget.download(item[1],tmp_file_name)
        except:
            pass